In [55]:
import os
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
from scipy import stats

In [56]:
dataPath = 'Data/2016-2019(One station)/shair-8781-1-6-1.csv'
airQualityData=pd.read_csv(dataPath, header=14,sep=';').rename(columns={'Start':'Start','Slut':'Stop'})
airQualityData.rename(columns = lambda x: re.sub('NOX.*','NOX',x), inplace = True)
airQualityData.rename(columns = lambda x: re.sub('PM10.*','PM10',x), inplace = True)
airQualityData.rename(columns = lambda x: re.sub('PM2.5.*','PM2_5',x), inplace = True)
airQualityData.rename(columns = lambda x: re.sub('NO2.*','NO2',x), inplace = True)
airQualityData.rename(columns = lambda x: re.sub('O3.*','O3',x), inplace = True)
airQualityData.rename(columns = lambda x: re.sub('Black Carbon.*','Black Carbon',x), inplace = True)
airQualityData['Start'] = pd.to_datetime(airQualityData['Start'])
airQualityData= airQualityData.drop('Stop',axis=1)
one_feature=airQualityData.drop(columns=['Black Carbon','NO2','NOX','O3','PM2_5'])
one_feature.head(5)

,Start,PM10
0,2015-01-01 00:00:00,28.4
1,2015-01-01 01:00:00,8.5
2,2015-01-01 02:00:00,8.7
3,2015-01-01 03:00:00,10.5
4,2015-01-01 04:00:00,12.1


In [76]:
train=one_feature[(one_feature['Start']<= "2018-12-31 23:00:00")]
test=one_feature[(one_feature['Start'] >= "2019-01-01 00:00:00")]

In [105]:
X_train = []
y_train = []
for i in range(5, len(training_set)):
    X_train.append(training_set[i-5:i])
    y_train.append(training_set[i])
    
X_train, y_train = np.array(X_train), np.array(y_train)
print (X_train.shape)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

(35059, 5)


In [81]:
index=[]
for i in range(0,len(X_train)):
    if np.isnan(X_train[i]).any() or np.isnan(y_train[i].any()):
        index.append(i)
X_train_new=np.delete(X_train,index)
y_train_new=np.delete(y_train,index)

In [60]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
31328/35059 [=========================>....] - ETA: 10s - loss: nan

KeyboardInterrupt: 

In [61]:
np.isnan(X_train).any()

True

In [82]:
len(X_train_new)

173488

In [84]:
X_train_new[1]

8.5

In [87]:
np.shape(X_train)

(35059, 5, 1)

In [99]:
len(X_train_new)/5

34697.6

In [102]:
X_train_new = np.reshape(X_train_new, (34697, 5, 1))

ValueError: cannot reshape array of size 173488 into shape (34690,5,1)

In [103]:
training_set[0:5]

array([28.4,  8.5,  8.7, 10.5, 12.1])